Biostat/Biomath M257 Homework 5

Due May 24 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [2]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 


Load packages:

In [3]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()
# Pkg.add("MathOptInterface")

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw5`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw5/Project.toml`
  [1e616198] COSMO v0.8.9
  [61c947e1] Clarabel v0.7.1
  [f65535da] Convex v0.15.4
  [a93c6f00] DataFrames v1.6.1
  [60bf3e95] GLPK v1.2.1
  [2e9cd046] Gurobi v1.2.3
  [87dc4568] HiGHS v1.9.0
  [b99e6be6] Hypatia v0.8.1
  [4076af6c] JuMP v1.22.1
  [67920dd8] KNITRO v0.14.2
  [b8f27783] MathOptInterface v1.29.0
  [1ec41992] MosekTools v0.15.1
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.3.1
  [c946c3f1] SCS v2.0.0
  [3eaba693] StatsModels v0.7.3


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to optimal design

Consider a linear model
\begin{aligned}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{aligned}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{aligned}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{aligned}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{aligned}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{aligned}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company asks you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $\mathbf{C}$ with each row a unique design point.

**Solution**

$\mathbf{C}$ has $6$ columns (intercept, A2, A3, B2, B3, B4) and $m = 3 \times 4 - 1 = 11$ rows when ignoring A3:B4. The row order follows the sequence seen in Q2:
$$
\begin{aligned}
\mathbf{C} = 
    \begin{bmatrix}
        \mathbf{x}_1^T\\
        \mathbf{x}_2^T\\
        \mathbf{x}_3^T\\
        \mathbf{x}_4^T\\
        \mathbf{x}_5^T\\
        \mathbf{x}_6^T\\
        \mathbf{x}_7^T\\
        \mathbf{x}_8^T\\
        \mathbf{x}_9^T\\
        \mathbf{x}_{10}^T\\
        \mathbf{x}_{11}^T
    \end{bmatrix}
    =
    \begin{bmatrix}
        1 & 0 & 0 & 0 & 0 & 0 \\
        1 & 1 & 0 & 0 & 0 & 0 \\
        1 & 0 & 1 & 0 & 0 & 0 \\
        1 & 0 & 0 & 1 & 0 & 0 \\
        1 & 1 & 0 & 1 & 0 & 0 \\
        1 & 0 & 1 & 1 & 0 & 0 \\
        1 & 0 & 0 & 0 & 1 & 0 \\
        1 & 1 & 0 & 0 & 1 & 0 \\
        1 & 0 & 1 & 0 & 1 & 0 \\
        1 & 0 & 0 & 0 & 0 & 1 \\
        1 & 1 & 0 & 0 & 0 & 1
    \end{bmatrix}
    \quad\text{with}\quad
    \mathbf{C}^T\mathbf{C} = \sum_{i=1}^{11}\mathbf{x}_i \mathbf{x}_1^T.
\end{aligned}
$$
Suppose $\mathbf{W}$ is a diagonal matrix with positive entries of $\mathbf{n}$ or $\mathbf{p}$, then we have
$$
\mathbf{V} = \left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)^{-1}
$$
with constraint $\mathbf{1}^T\mathbf{n} = n$ or $\mathbf{1}^T\mathbf{p} = 1$.

<!-- \begin{array}{c|ccccc}
    & \text{Intercept} & \text{A2} & \text{A3} & \text{B2} & \text{B3} & \text{B4} \\
    \hline
    \text{A1:B1} & 1 & 0 & 0 & 0 & 0 & 0 \\
    \text{A2:B1} & 1 & 1 & 0 & 0 & 0 & 0 \\
    \text{A3:B1} & 1 & 0 & 1 & 0 & 0 & 0 \\
    \text{A1:B2} & 1 & 0 & 0 & 1 & 0 & 0 \\
    \text{A2:B2} & 1 & 1 & 0 & 1 & 0 & 0 \\
    \text{A3:B2} & 1 & 0 & 1 & 1 & 0 & 0 \\
    \text{A1:B3} & 1 & 0 & 0 & 0 & 1 & 0 \\
    \text{A2:B3} & 1 & 1 & 0 & 0 & 1 & 0 \\
    \text{A3:B3} & 1 & 0 & 1 & 0 & 1 & 0 \\
    \text{A1:B4} & 1 & 0 & 0 & 0 & 0 & 1 \\
    \text{A2:B4} & 1 & 1 & 0 & 0 & 0 & 1
\end{array} -->

In [1]:
C = [1 0 0 0 0 0;
     1 1 0 0 0 0;
     1 0 1 0 0 0;
     1 0 0 1 0 0;
     1 1 0 1 0 0;
     1 0 1 1 0 0;
     1 0 0 0 1 0;
     1 1 0 0 1 0;
     1 0 1 0 1 0;
     1 0 0 0 0 1;
     1 1 0 0 0 1]

11×6 Matrix{Int64}:
 1  0  0  0  0  0
 1  1  0  0  0  0
 1  0  1  0  0  0
 1  0  0  1  0  0
 1  1  0  1  0  0
 1  0  1  1  0  0
 1  0  0  0  1  0
 1  1  0  0  1  0
 1  0  1  0  1  0
 1  0  0  0  0  1
 1  1  0  0  0  1

## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got (using Hypatia solver), which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.271 │   0.200 │   8.000 │  27.000 │  20.000 │
│      A2B1 │   0.082 │   0.150 │   0.101 │   8.000 │  15.000 │  10.000 │
│      A3B1 │   0.097 │   0.118 │   0.104 │  10.000 │  12.000 │  10.000 │
│      A1B2 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B2 │   0.082 │   0.037 │   0.051 │   8.000 │   4.000 │   5.000 │
│      A3B2 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B3 │   0.082 │   0.037 │   0.051 │   8.000 │   3.000 │   5.000 │
│      A3B3 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B4 │   0.109 │   0.079 │   0.106 │  11.000 │   8.000 │  10.000 │
│      A2B4 │   0.109 │   0.075 │   0.080 │  11.000 │   7.000 │   8.000 │
│       Obj │   8.987 │  13.000 │  38.925 │   8.988 │  13.028 │  38.946 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

**Solution**

**$D$-optimal design**:

It is important to note that
$$
\begin{aligned}
	&\text{minimize} \quad \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}
	= \left[\det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)\right]^{-1}\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad -\det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad -\text{logdet} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)
	= -\text{logdet}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right).
\end{aligned}
$$
So, we don't need to take inverse of $\mathbf{V}$. I use `minimize` (instead of `maximize`) to obtain a positive objective function, as Dr. Zhou did.

In [4]:
using LinearAlgebra, Convex

In [80]:
m = size(C, 1) # 11
p = Variable(m, Positive())
V⁻¹ = transpose(C) * Diagonal(p) * C
problem = minimize(-logdet(V⁻¹))
# problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem

minimize
└─ - (convex; real)
   └─ logdet (concave; real)
      └─ * (affine; positive)
         ├─ …
         └─ …
subject to
└─ == constraint (affine)
   ├─ sum (affine; positive)
   │  └─ 11-element positive variable (id: 878…751)
   └─ 1

status: `solve!` not called yet

I use the Hypatia solver, as Dr. Zhou did.

In [81]:
using Hypatia, MathOptInterface

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check optimal value and optimal solution
D_opt_obj = round(problem.optval, digits = 3)
D_opt_p = round.(p.value, digits = 3)
println("Obj = ", D_opt_obj)
vcat(D_opt_p, [D_opt_obj])

  0.080410 seconds (40.67 k allocations: 3.441 MiB)
Obj = 8.987


12×1 Matrix{Float64}:
 0.082
 0.082
 0.097
 0.082
 0.082
 0.097
 0.082
 0.082
 0.097
 0.109
 0.109
 8.987

In [63]:
# check constraint satisfication
sum(p.value) ≈ 1.0

true

In [105]:
# convert p to integer n
D_opt_n = round.(p.value * 100, digits = 0)
display(D_opt_n)
D_opt_obj_n = -logdet(transpose(C) * Diagonal(vec(D_opt_n)/100) * C)
D_opt_obj_n = round(D_opt_obj_n, digits = 3)

11×1 Matrix{Float64}:
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
 11.0
 11.0

8.988

**$E$-optimal design**

Use the equivalent objective function to avoid the inverse as shown below:
$$
\begin{aligned}
	&\text{minimize} \quad\lambda_{\text{max}}(\mathbf{V})\\
	\Longleftrightarrow\quad
	&\text{maximize} \quad \lambda_{\text{min}} \left(\mathbf{V}^{-1} \right)
	= \lambda_{\text{min}}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad 1/\lambda_{\text{min}}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right).
\end{aligned}
$$

In [106]:
problem = minimize(1 / eigmin(V⁻¹))
# problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check the status, optimal value
E_opt_obj = round(problem.optval, digits = 3)
E_opt_p = round.(p.value, digits = 3)
println("Obj = ", E_opt_obj)
vcat(E_opt_p, [E_opt_obj])

  0.032508 seconds (16.30 k allocations: 1.162 MiB)
Obj = 13.0


12×1 Matrix{Float64}:
  0.258
  0.162
  0.118
  0.054
  0.041
  0.059
  0.054
  0.041
  0.059
  0.074
  0.08
 13.0

In [12]:
sum(p.value) ≈ 1.0

true

To convert $\mathbf{p}$ to integer $\mathbf{n}$ with $n=100$ by rounding and recalculate the objective function.

In [109]:
E_opt_n = round.(p.value * 100, digits = 0)
display(E_opt_n)
E_opt_obj_n = 1 / eigmin(transpose(C) * Diagonal(vec(E_opt_n) / 100) * C)
E_opt_obj_n = round(E_opt_obj_n, digits = 3)

11×1 Matrix{Float64}:
 26.0
 16.0
 12.0
  5.0
  4.0
  6.0
  5.0
  4.0
  6.0
  7.0
  8.0

13.152

Note that although
$$
\begin{aligned}
	&\text{minimize} \quad 1/\lambda_{\text{min}}\left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)\\
	\Longleftrightarrow\quad
	&\text{minimize} \quad - \lambda_{\text{min}} \left(\mathbf{C}^T\mathbf{W}\mathbf{C}\right)
\end{aligned},
$$
the results are not completely the same and closer to what Dr. Zhou obtained.

In [110]:
problem = minimize(- eigmin(V⁻¹))
# problem.constraints += p .>= 0
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check the status, optimal value
E_opt_obj2 = round(problem.optval, digits = 3)
E_opt_p2 = round.(p.value, digits = 3)
println("Obj = ", E_opt_obj2)
vcat(E_opt_p2, [E_opt_obj2])

  0.045014 seconds (10.55 k allocations: 890.797 KiB)
Obj = -0.077


12×1 Matrix{Float64}:
  0.273
  0.152
  0.114
  0.057
  0.039
  0.057
  0.057
  0.039
  0.057
  0.081
  0.073
 -0.077

with an equivalent objective function, i.e., $-(-0.077)^{-1} = 13$.

In [113]:
# convert p to integer n
E_opt_n2 = round.(p.value * 100, digits = 0)
display(E_opt_n2)
E_opt_obj_n2 = 1 / eigmin(transpose(C) * Diagonal(vec(E_opt_n2) / 100) * C)
E_opt_obj_n2 = round(E_opt_obj_n2, digits = 3)

11×1 Matrix{Float64}:
 27.0
 15.0
 11.0
  6.0
  4.0
  6.0
  6.0
  4.0
  6.0
  8.0
  7.0

13.016

**$A$-optimal design**:

Use `trace_mpower(A, t, C)` $= \text{tr}(\mathbf{C}\mathbf{A}^t)$.

In [118]:
problem = minimize(trace_mpower(V⁻¹, -1, Matrix{Float64}(I, 6, 6)))
# problem.constraints += p .>= 0 # not needed using Positive()
problem.constraints += sum(p) == 1
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check the status, optimal value
A_opt_obj = round(problem.optval, digits = 3)
A_opt_p = round.(p.value, digits = 3)
println("Obj = ", A_opt_obj)
vcat(A_opt_p, [A_opt_obj])

  0.065479 seconds (59.27 k allocations: 3.982 MiB)
Obj = 38.925


12×1 Matrix{Float64}:
  0.2
  0.101
  0.104
  0.086
  0.051
  0.068
  0.086
  0.051
  0.068
  0.106
  0.08
 38.925

In [70]:
# check constraint satisfication
sum(p.value) ≈ 1.0

true

In [122]:
# convert p to integer n
A_opt_n = round.(p.value * 100, digits = 0)
display(A_opt_n)
A_opt_obj_n = trace_mpower(transpose(C) * Diagonal(vec(A_opt_n) / 100) * C, -1, Matrix{Float64}(I, 6, 6))
A_opt_obj_n = round(A_opt_obj_n, digits = 3)

11×1 Matrix{Float64}:
 20.0
 10.0
 10.0
  9.0
  5.0
  7.0
  9.0
  5.0
  7.0
 11.0
  8.0

38.553

Let's merge the three results in table.

In [123]:
using DataFrames
treatment = [
    "A1B1", "A2B1", "A3B1",
    "A1B2", "A2B2", "A3B2",
    "A1B3", "A2B3", "A3B3",
    "A1B4", "A2B4", "Obj"
]

df_Q2 = DataFrame(
    design_pt = treatment,
    D_opt   = vcat(D_opt_p,  [D_opt_obj])[:],
    E_opt   = vcat(E_opt_p,  [E_opt_obj])[:],
    E_opt2  = vcat(E_opt_p2, [- 1 / E_opt_obj2])[:],
    A_opt   = vcat(A_opt_p,  [A_opt_obj])[:],
    D_opt_n = vcat(D_opt_n,  [D_opt_obj_n])[:],
    E_opt_n = vcat(E_opt_n,  [E_opt_obj_n])[:],
    E_opt_n2 = vcat(E_opt_n2, [E_opt_obj_n2])[:],
    A_opt_n = vcat(A_opt_n,  [A_opt_obj_n])[:]
)

Row,design_pt,D_opt,E_opt,E_opt2,A_opt,D_opt_n,E_opt_n,E_opt_n2,A_opt_n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A1B1,0.082,0.258,0.273,0.2,8.0,26.0,27.0,20.0
2,A2B1,0.082,0.162,0.152,0.101,8.0,16.0,15.0,10.0
3,A3B1,0.097,0.118,0.114,0.104,10.0,12.0,11.0,10.0
4,A1B2,0.082,0.054,0.057,0.086,8.0,5.0,6.0,9.0
5,A2B2,0.082,0.041,0.039,0.051,8.0,4.0,4.0,5.0
6,A3B2,0.097,0.059,0.057,0.068,10.0,6.0,6.0,7.0
7,A1B3,0.082,0.054,0.057,0.086,8.0,5.0,6.0,9.0
8,A2B3,0.082,0.041,0.039,0.051,8.0,4.0,4.0,5.0
9,A3B3,0.097,0.059,0.057,0.068,10.0,6.0,6.0,7.0


## Q3 (30 pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial **with $n=100$**.

Hint: This is what I got using Pajarito with Gurobi as the `oa_solver` and Mosek as the `conic_solver`.

```
Exact Optimal Design
┌───────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │
│    String │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┤
│      A1B1 │   8.000 │  34.000 │  20.000 │
│      A2B1 │   8.000 │  10.000 │  10.000 │
│      A3B1 │  10.000 │  10.000 │  10.000 │
│      A1B2 │   8.000 │   9.000 │   9.000 │
│      A2B2 │   8.000 │   0.000 │   5.000 │
│      A3B2 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   8.000 │   8.000 │   9.000 │
│      A2B3 │   8.000 │   4.000 │   5.000 │
│      A3B3 │  10.000 │   4.000 │   7.000 │
│      A1B4 │  11.000 │   9.000 │  10.000 │
│      A2B4 │  11.000 │   6.000 │   8.000 │
│       Obj │   8.988 │  13.011 │  38.946 │
└───────────┴─────────┴─────────┴─────────┘
```

In [130]:
using Pajarito, SCS

In [128]:
n = Variable(m, IntVar)
V⁻¹ = transpose(C) * Diagonal(n) * C

problem = minimize(trace_mpower(V⁻¹, -1, Matrix{Float64}(I, 6, 6)))
problem.constraints += sum(n) == 100
problem

const MOI = MathOptInterface
solver = Hypatia.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("verbose"), 0)

@time solve!(problem, solver)

# Check the status, optimal value
A_opt_obj = round(problem.optval, digits = 3)
A_opt_p = round.(p.value, digits = 3)
println("Obj = ", A_opt_obj)
vcat(A_opt_p, [A_opt_obj])

MathOptInterface.UnsupportedConstraint{MathOptInterface.VariableIndex, MathOptInterface.Integer}: MathOptInterface.UnsupportedConstraint{MathOptInterface.VariableIndex, MathOptInterface.Integer}: `MathOptInterface.VariableIndex`-in-`MathOptInterface.Integer` constraint is not supported by the model.

## Q4 (30 bonus points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.